# Kubernetes Storage & `StatefulSets`

Kubernetes has transformed the way we deploy and manage applications. While it excels at handling stateless workloads, dealing with stateful applications introduces unique challenges. In this lesson, we will delve into two critical aspects of Kubernetes that address these challenges: *Kubernetes Storage* and *`StatefulSets`*.

## Overview of Kubernetes Storage

Managing data in Kubernetes requires careful consideration. Kubernetes offers various storage options to enable data persistence across containerized applications. From *Persistent Volumes (PVs)* to *Persistent Volume Claims (PVCs)*, understanding these concepts is essential for ensuring data reliability and availability in your applications. We will cover these concepts in the next section in detail.

## Overview of `StatefulSets`

`StatefulSets`, a specialized controller in Kubernetes, are designed to handle stateful workloads with precision. They provide ordered, predictable deployment and scaling of pods, making them ideal for applications like databases, caching systems, and more. 

## Volumes in Kubernetes

Volumes in Kubernetes play a crucial role in managing data within containers, but they differ in several ways from their counterparts in Docker. Let's explore these differences and dive into the high-level features that make Kubernetes Volumes a powerful tool for managing data in containerized applications.

### Volumes in Docker

In Docker, volumes are directories located on the host machine's disk or even within another container. These volumes can be mounted into containers during runtime, facilitating data sharing between containers or even across different host machines using drivers. While Docker volumes are valuable, they have limitations, especially when it comes to large-scale deployments.

### Key Differentiators of Kubernetes Volumes

Kubernetes Volumes offer a more versatile and robust solution for managing data in containerized applications. Here are some high-level features that set Kubernetes Volumes apart:

1. **Simultaneous Mounting**: Kubernetes Volumes can be mounted simultaneously into multiple containers within the same Pod. This capability is particularly useful when you have multiple containers that need access to the same data.

2. **Ephemeral and Persistent Volumes**: In Kubernetes, Volumes can be categorized as either ephemeral or persistent. *Ephemeral Volumes* have lifetimes tied to their Pods. When the Pod restarts or is rescheduled to a different node, ephemeral Volumes are recreated. On the other hand, *persistent Volumes* have lifetimes independent of their Pods. They offer durability and data retention even when Pods come and go.

3. **Automatic Data Availability**: Kubernetes takes care of ensuring that data is available across container restarts. This functionality is handled by `kubelet`, the Kubernetes node agent. It ensures that data remains accessible as long as the associated Volume exists, regardless of the Pod's state.

### Attaching Volumes to Pods

Volumes can be attached to Pods using the following specifications:

- `.spec.volumes`: This field specifies which Volumes to use in the Pod
- `.spec.containers[*].volumeMounts`: Within a specific container's configuration, you can define where and which Volume(s) to mount. This offers fine-grained control over data access for each container in the Pod.

Consider the below example using a `DaemonSet`:

``` yaml
apiVersion: apps/v1
kind: DaemonSet
metadata:
  name: fluentd-elasticsearch
  labels:
    k8s-app: fluentd-logging
spec:
  selector:
    matchLabels:
      name: fluentd-elasticsearch
  template:
    metadata:
      labels:
        name: fluentd-elasticsearch
    spec:
      containers:
      - name: fluentd-elasticsearch
        image: quay.io/fluentd_elasticsearch/fluentd:v2.5.2
        resources:
          limits:
            memory: 200Mi
          requests:
            cpu: 100m
            memory: 200Mi
        # Here we can mount them with `name` matching
        # Ephemeral Volumes
        volumeMounts:
        - name: varlog
          mountPath: /var/log
        - name: varlibdockercontainers
          mountPath: /var/lib/docker/containers
          readOnly: true
      terminationGracePeriodSeconds: 30
      # Here we define our volumes
      # Data from POD will be mounted
      volumes:
      - name: varlog
        hostPath:
          path: /var/log
      - name: varlibdockercontainers
        hostPath:
          path: /var/lib/docker/container
```

### Hands-On: Attaching Volumes to Pods

1. Create a `YAML` file for the `DaemonSet` above
2. Use the `describe` command from `kubectl` to observe the volumes
3. Start the Minikube dashboard by running `minikube dashboard`, and explore the volumes in the Minikube dashboard.

In [2]:
# Create the DaemonSet
!kubectl 

daemonset.apps/fluentd-elasticsearch created


In [5]:
# Use the describe command after finding out the name of the pod
!kubectl 

Name:             fluentd-elasticsearch-2ctkd
Namespace:        default
Priority:         0
Service Account:  default
Node:             minikube/192.168.49.2
Start Time:       Mon, 11 Sep 2023 14:54:41 +0100
Labels:           controller-revision-hash=6499fb66c6
                  name=fluentd-elasticsearch
                  pod-template-generation=1
Annotations:      <none>
Status:           Running
IP:               10.244.0.103
IPs:
  IP:           10.244.0.103
Controlled By:  DaemonSet/fluentd-elasticsearch
Containers:
  fluentd-elasticsearch:
    Container ID:   docker://5d24e634937b4e85b23861c092fa1d2056ebed4a8265b9608f7cd4826a3782ec
    Image:          quay.io/fluentd_elasticsearch/fluentd:v2.5.2
    Image ID:       docker-pullable://quay.io/fluentd_elasticsearch/fluentd@sha256:aec118bb3d1c4af358c1d495b14c12781c4ab5e8cfb455edb1ebd2e92750e31d
    Port:           <none>
    Host Port:      <none>
    State:          Running
      Started:      Mon, 11 Sep 2023 14:55:47 +0100
    Rea

Now run `minikube dashboard`. The dashboard should have an appearance similar to that in the figure below, with the volumes mounted in the container:
<p><img src=images/volume.png></p>

## Persistent Volumes

Kubernetes provides two key resources for managing persistent storage: `PersistentVolume` and `PersistentVolumeClaim`. These resources abstract the provisioning and utilization of storage in your cluster.

> A `PersistentVolume` represents a storage volume in a cluster, provisioned by a cluster administrator. 

These volumes are similar to physical storage devices found on the host machine. Importantly, they exist independently of any specific pod's lifecycle. When bound to a pod, they function similarly to regular volumes, offering a reliable means of data storage.

> In contrast, a `PersistentVolumeClaim` is a user's request for the platform to create a `PersistentVolume` on their behalf. 

Conceptually, PVCs share similarities with pods in the sense that they consume resources. However, while pods request resources like CPU and RAM, PVCs specify storage requirements, including size and access modes (e.g., `read-write` or `read-only`).
    
### Provisioning `PersistentVolume`s

The provisioning of PersistentVolumes can occur through two primary methods:

- **Static Provisioning**: In this approach, cluster administrators pre-create PVs that are available for users to consume. Users then select a PV that matches their needs.

- **Dynamic Provisioning**: Dynamic provisioning is a more automated approach. When a user requests a PVC, the cluster attempts to dynamically provision an appropriate PV based on the PVC's requirements and `StorageClasses`. Dynamic provisioning ensures that PVs always match the requirements specified by PVCs.

### Reclaim Policy

Once a user is finished with a volume, they can initiate the deletion of the associated PVC, which subsequently triggers the reclamation of the underlying resource. Three reclaim policies govern how this resource reclamation occurs:

- **Retain**: The "retain" policy leaves the data intact, preserving both the `PersistentVolume` and any external storage

- **Delete**: The "delete" policy removes not only the `PersistentVolume` but also the external storage associated with it. This policy is the default for dynamic provisioning, though it can be configured.

- **Recycle** (Deprecated): The "recycle" policy has been deprecated in favor of dynamic provisioning. Dynamic provisioning is now the recommended approach for creating and managing `PersistentVolumes`.

`PersistentVolumes` can exist in one of four states:

- **Available**: In this state, the `PersistentVolume` is ready to be bound to a pod, but it is not yet associated with any pod

- **Bound**: When a `PersistentVolume` is bound, it means that a PVC has been successfully associated with it, and it is actively serving a pod

- **Released**: The "released" state occurs when a `PersistentVolumeClaim` is deleted, but the resource has not yet been reclaimed by the cluster.

- **Failed**: In the event that reclamation of a volume fails, it is marked as "failed." This can happen due to various reasons, such as an issue with the storage system.

### Specifying `PersistentVolume` Configuration

Similar to pods and other Kubernetes resources, PV objects are defined using `.yaml` configuration files. Below is an example configuration file illustrating the key attributes you can specify when creating a `PersistentVolume`:

``` yaml
apiVersion: v1
kind: PersistentVolume
metadata:
  name: task-pv-volume
  labels:
    type: local
spec:
  storageClassName: manual
  capacity:
    storage: 10Gi
  accessModes:
    - ReadWriteOnce
  hostPath:
    path: "/mnt/data"
```

#### Common Attributes

1. `.spec.capacity.storage`: In this example, we request 10GB of storage. The storage attribute allows you to specify the desired storage capacity. It's important to note that currently, only storage capacity can be requested. Kubernetes uses its resource model, which defines storage in units, to interpret the requested capacity.

2. `.spec.volumeMode`: This attribute allows you to choose between two modes: FileSystem (default) or Block
    - **FileSystem**: Represents a directory that can be mounted in pods, and it's the most commonly used mode

    - **Block**: Utilizes a raw block of storage without creating a filesystem on it. The Block mode is less common because applications need to know how to access raw data directly. 

3. `.spec.accessModes`: This attribute specifies how data can be accessed by pods using the `PersistentVolume`. There are three common access modes:
    - `ReadWriteOnce`: Allows a single Node to mount the volume with read-write access. This mode is suitable for scenarios where only only one pod needs read-write access to the data.

    - `ReadOnlyMany`: Permits multiple Nodes to mount the volume, but only read access is granted. Useful for scenarios where multiple pods need read-only access to the data.

    - `ReadWriteMany`: Similar to `ReadOnlyMany`, but grants read-write access to multiple nodes. Applications using this mode should handle possible data races.
    
    - `ReadWriteOncePod`: Provides read-write access but only for a single pod

Access modes may behave differently depending on the type of `PersistentVolume` provider, and they should be chosen based on your application's requirements. A few of them are shown below:

![](./images/modes_providers.png)

4. `.spec.storageClassName`: This attribute specifies the `StorageClass` associated with the `PersistentVolume`. If left unspecified, no `StorageClass` will be assigned to the `PersistentVolume`.

5. `.spec.mountOptions`: This attribute is not supported by all types of `PersistentVolumes`. It specifies how the volume should be mounted, and in most cases, it can be left as is without the need for customization.

#### Hands-On

1. Create a `.yaml` file with the above configuration for creating a `PersistentVolume` resource
2. Apply the configuration using `kubectl` to create the `PersistentVolume` resource within your Kubernetes cluster
3. Observe the status and properties of the `PersistentVolume` to ensure it's successfully provisioned and available for use

In [8]:
# apply the configuration
!kubectl

persistentvolume/task-pv-volume created


In [11]:
# observe tha status of the volume
!kubectl

NAME             CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS      CLAIM   STORAGECLASS   REASON   AGE
task-pv-volume   10Gi       RWO            Retain           Available           manual                  13m


## StorageClasses

> StorageClasses provide a flexible way for administrators to define the classes of storage they offer. StorageClasses enable the dynamic provisioning of storage and serve as templates for creating new `PersistentVolumes`.

### Defining a `StorageClass`

StorageClasses are typically defined using `.yaml` configuration files, and they can be referenced by PersistentVolume configuration files. Let's consider an example of a `StorageClass` definition:

```yaml
apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: standard
provisioner: kubernetes.io/aws-ebs
parameters:
  type: gp2
reclaimPolicy: Retain
allowVolumeExpansion: true
mountOptions:
  - debug
volumeBindingMode: Immediate
```

### Mandatory Fields

- `metadata.name`: The `name` field allows users to request the use of this specific `StorageClass` when creating `PersistentVolumes`

- `provisioner`: This field specifies the volume plugin used for provisioning `PersistentVolumes`. Commonly used provisioners are shipped with Kubernetes under the `kubernetes.io` prefix. For example:
  - `local`: `kubernetes.io/no-provisioner` (used for creating PersistentVolumes dynamically from local resources)

```yaml
apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
name: local-storage
provisioner: kubernetes.io/no-provisioner
volumeBindingMode: WaitForFirstConsumer
```

  - `GCEPersistentDisk`: Used for provisioning persistent disks from Google Cloud

```yaml
apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: slow
provisioner: kubernetes.io/gce-pd
parameters:
  type: pd-standard
  fstype: ext4
  replication-type: none
```

Importantly, Kubernetes isn't limited to these built-in provisioners. Providers can create their custom provisioners to cater to specific storage solutions.

- `parameters`: This field allows for the definition of per-provisioner specifications of volume properties. Examples of different internal provisioners and their parameters can be found in the Kubernetes documentation.

- `reclaimPolicy`: This field specifies what should happen to the created `PersistentVolume` once it's no longer bound to a `PersistentVolumeClaim`. Options typically include Delete or Retain, depending on whether you want to delete or retain the storage resources associated with the `PersistentVolume`.

> StorageClasses play a pivotal role in enabling dynamic provisioning of storage resources in Kubernetes. They allow administrators to define storage characteristics and policies while providing developers with a convenient way to request storage resources tailored to their application's needs.

## Expandable Volumes

Starting with Kubernetes version `1.11`, the ability to dynamically expand volumes was introduced. This feature enables the adjustment of storage capacity in response to changing requirements by modifying the `PersistentVolumeClaim` (PVC) and applying a new configuration.

> The easiest way to take advantage of this dynamic volume expansion capability is to use an internal cloud provider, and the process is straightforward: you simply need to set `.allowVolumeExpansion: true` in the `StorageClass` definition.

For example:

```yaml
apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: standard
provisioner: kubernetes.io/aws-ebs
parameters:
  type: gp2
reclaimPolicy: Retain
allowVolumeExpansion: true
mountOptions:
  - debug
volumeBindingMode: Immediate
```

Many internal cloud providers support expandable volumes. Here is a list of some commonly used providers:

![](./images/expandable-volumes.png)

## Storage Summary

This summary provides a guideline for managing storage in Kubernetes:

- **Data Sharing Consideration**:

  - Determine whether your data needs to be shared between containers or pods

  - If sharing data between containers within the same pod, consider using ephemeral volumes for simple data exchange between applications

  - If data needs to be shared between pods or preserved after pod termination, use `PersistentVolume`s

- **Dynamic Provisioning Decision**:

  - Decide whether dynamic provisioning is required. This is particularly useful when it's challenging to predict the exact storage requirements in advance.

- **Without Dynamic Provisioning**:

  - Create a `PersistentVolume` `.yaml` configuration file (defines how a volume is created)

  - Create a `PersistentVolumeClaim` `.yaml` configuration file (specifies how a volume is requested)

  - Create a `MyApplication .yaml` configuration file (the workload resource). Avoid using bare pods in your configurations.

- **With Dynamic Provisioning**:

  - Create a `StorageClass` `.yaml` configuration file (acts as a template for providing `PersistentVolume`s to pods as needed).

- **Expandable Volumes**:
  - Consider using expandable volumes when you know the maximum number of pods that will run at any given time and the exact amount of storage required for each pod is unknown

Dynamic provisioning is beneficial for large-scale applications, where manual provisioning is impractical due to the following reasons:
  - A high number of pods are requesting significant storage allocations

  - It's challenging to predict in advance how many pods will run

> In large deployments, cloud-storage providers are often preferred over local storage, as local storage may prove insufficient.

## `StatefulSets`
   
In Kubernetes, workloads are typically used for stateless applications, where data is not written to external storage. However, when it comes to managing stateful applications, Kubernetes provides a powerful resource called `StatefulSets`.

> `StatefulSets` are a specific type of workload resource in Kubernetes that adds a storage volume to each pod it manages. This unique characteristic allows all data associated with a pod to be stored persistently. This is particularly useful for applications like databases, where data persistence is critical between pod restarts.

### Limitations

- **Storage Provisioning**: The storage used by `StatefulSets` must be provisioned either by a cluster administrator or dynamically using `StorageClass`es

- **Storage Preservation**: Deleting a `StatefulSet` does not automatically delete the associated storage. Storage preservation is prioritized to prevent accidental data loss.

- **Headless Service Requirement**: `StatefulSets` require the presence of a headless Service. A headless Service is used to manage the network identity of pods, providing stable DNS names for each pod.

### Components of a `StatefulSet`

To create a `StatefulSet`, you'll need specific `.yaml` definitions. Here's an overview of the necessary components:

1. Headless Service

```yaml
apiVersion: v1
kind: Service
metadata:
  name: nginx
  labels:
    app: nginx
spec:
  ports:
  - port: 80
    name: web
  clusterIP: None
  selector:
    app: nginx
```

2. `StatefulSet`

```yaml
apiVersion: apps/v1
kind: StatefulSet
metadata:
  name: web
spec:
  serviceName: "nginx"
  replicas: 2
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: k8s.gcr.io/nginx-slim:0.8
        ports:
        - containerPort: 80
          name: web
        volumeMounts:
        - name: www
          mountPath: /usr/share/nginx/html
  volumeClaimTemplates:
  - metadata:
      name: www
    spec:
      accessModes: ["ReadWriteOnce"]
      resources:
        requests:
          storage: 1Gi
```

In this example:

- A headless Service named `nginx` is created to manage the network domain

- The `StatefulSet` named `web` specifies that two replicas of the `nginx` container will be launched in unique pods

- `volumeClaimTemplates` are used to provide stable storage using PersistentVolumes provisioned by a `PersistentVolume` provisioner

### Hands-On

1. Create a `.yaml` file with the above configuration for creating a `StatefulSet` and its associated resources

2. Apply the configuration using `kubectl` to create the `StatefulSet` and related objects within your Kubernetes cluster

3. Access the Kubernetes dashboard and observe the `PersistentVolumeClaims` (PVCs). You should see that the volumes are already bound and available for use.

In [15]:
!kubectl

service/nginx created
statefulset.apps/web created


The dashboard's appearance should be similar to that shown below:

<p align=center><img src=images/PVC.png></p>

## Key Takeaways

- Volumes in Kubernetes provide a way to store and share data within containers. Volumes can be mounted simultaneously, making it easy to share data between containers within the same pod.
- Persistent Volumes (PVs) are physical storage volumes in the cluster provisioned by administrators, while Persistent Volume Claims (PVCs) are requests for storage made by pods
- `StorageClass`es define the classes of storage offered by administrators and can be used to dynamically provision storage Different provisioners can be used with `StorageClass`es, enabling flexibility in choosing storage solutions.
- Dynamic provisioning allows Kubernetes to automatically create PVs based on PVC requirements, making it suitable for large-scale deployments. It eliminates the need for manual provisioning and ensures that storage matches the demands of pods.
- Expandable volumes can be used when the maximum number of pods running simultaneously is known, but the exact storage requirements per pod are uncertain. 
- `StatefulSets` are workload resources used for stateful applications that require persistent storage. They manage pods with storage volumes, ensuring data persistence even across pod restarts.
- `StatefulSets` require a headless Service to manage network identity
- Storage provisioning for `StatefulSets` must be done by an administrator or dynamically using `StorageClass`es
- Deleting a StatefulSet does not automatically delete associated storage to preserve data